In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import math
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.utils import shuffle
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

np.random.seed(42)

In [2]:
iris_data = load_iris() 
x = iris_data.data
y_ = iris_data.target.reshape(-1, 1)

encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y_)

#print(y)

# Split the data for training and testing
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.20)

/home/havish/.local/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [3]:
def relu(x):
    x[x<0]=0
    return x
def softmax(arr):
#     arr = arr/np.max(arr)
    return np.exp(arr)/(np.sum(np.exp(arr),axis=0))
def diff_relu(arr):
    z = np.zeros(arr.shape)
    z[arr<=0] = 0
    z[arr>0] = 1
    return z

In [4]:
def initializer(fan_out,fan_in):
    limit = np.sqrt(2*1.0/(fan_in+fan_out))
#     return np.random.uniform(-limit,limit,(fan_out,fan_in))
    return np.random.normal(0,limit,(fan_out,fan_in))

In [5]:
### architecture ###
in_dim = 4
hid1_dim = 10
hid2_dim = 10
out_dim = 3
W1 = initializer(hid1_dim,in_dim)
b1 = initializer(hid1_dim,1)
W2 = initializer(hid2_dim,hid1_dim)
b2 = initializer(hid2_dim,1)
W3 = initializer(out_dim,hid2_dim)
b3 = initializer(out_dim,1)

In [6]:
epochs = 500
num_samples = len(train_x)
batch_size = 1
delta_const = 1e-10
num_batches = num_samples/batch_size
eps = 1e-3
rho = 0.4
r = {"W1" : np.zeros(W1.shape) , "W2" : np.zeros(W2.shape) ,"W3" : np.zeros(W3.shape),"b1": np.zeros(b1.shape)
     ,"b2" :np.zeros(b2.shape) , "b3" : np.zeros(b3.shape)}
for i in range(epochs):
    (x_train_subs,y_train_subs) = shuffle(train_x,train_y,random_state = 40)
    loss = 0
    for j in range(int(num_batches)):
        W1_upd = np.zeros((hid1_dim,in_dim))
        b1_upd = np.zeros((hid1_dim,1))
        W2_upd = np.zeros((hid2_dim,hid1_dim))
        b2_upd = np.zeros((hid2_dim,1))
        W3_upd = np.zeros((out_dim,hid2_dim))
        b3_upd = np.zeros((out_dim,1))
        for k in range(batch_size):
            z1 = relu(np.matmul(W1,x_train_subs[j]).reshape(-1,1)+b1)
        
            z2 = relu(np.matmul(W2,z1).reshape(-1,1)+b2)

            out = softmax(np.matmul(W3,z2).reshape(-1,1) + b3)
        
            loss = loss + -np.log(out[np.argmax(y_train_subs[j])])
        
            del_3 = out - y_train_subs[j].reshape(-1,1)
            del_2 = np.matmul(W3.T,del_3)*diff_relu(z2)
            del_1 = np.matmul(W2.T,del_2)*diff_relu(z1)

            b3_upd += del_3
#         b3_upd = b3_upd.reshape(len(b3),1)
            b2_upd += del_2
#         b2_upd = b2_upd.reshape(len(b2),1)
            b1_upd += del_1
#         b1_upd = b1_upd.reshape(len(b1),1)
            W3_upd += np.matmul(del_3,z2.T)
            W2_upd += np.matmul(del_2,z1.T)
            W1_upd += np.matmul(del_1,x_train_subs[j].reshape(-1,1).T)
        r["W1"] = (1-rho)*r["W1"] + rho*W1_upd*W1_upd
        r["W2"] = (1-rho)*r["W2"] + rho*W2_upd*W2_upd
        r["W3"] = (1-rho)*r["W3"] + rho*W3_upd*W3_upd
        r["b1"] = (1-rho)*r["b1"] + rho*b1_upd*b1_upd
        r["b2"] = (1-rho)*r["b2"] + rho*b2_upd*b2_upd
        r["b3"] = (1-rho)*r["b3"] + rho*b3_upd*b3_upd
        W3 = W3 - (eps*W3_upd)/np.sqrt(delta_const + r["W3"])
        W2 = W2 - (eps*W2_upd)/np.sqrt(delta_const + r["W2"])
        W1 = W1 - (eps*W1_upd)/np.sqrt(delta_const + r["W1"])
        b3 = b3 - (eps*b3_upd)/np.sqrt(delta_const + r["b3"])
        b2 = b2 - (eps*b2_upd)/np.sqrt(delta_const + r["b2"])
        b1 = b1 - (eps*b1_upd)/np.sqrt(delta_const + r["b1"])
        
    print("Epoch: " + str(i) + " Loss: " + str(loss/num_samples))

Epoch: 0 Loss: [1.7473769]
Epoch: 1 Loss: [1.12497395]
Epoch: 2 Loss: [0.69787358]
Epoch: 3 Loss: [0.47194844]
Epoch: 4 Loss: [0.3929087]
Epoch: 5 Loss: [0.35021514]
Epoch: 6 Loss: [0.31959457]
Epoch: 7 Loss: [0.29650163]
Epoch: 8 Loss: [0.26944105]
Epoch: 9 Loss: [0.2394433]
Epoch: 10 Loss: [0.21526301]
Epoch: 11 Loss: [0.19657442]
Epoch: 12 Loss: [0.18108616]
Epoch: 13 Loss: [0.16836384]
Epoch: 14 Loss: [0.15769355]
Epoch: 15 Loss: [0.148876]
Epoch: 16 Loss: [0.14146061]
Epoch: 17 Loss: [0.13538122]
Epoch: 18 Loss: [0.1302603]
Epoch: 19 Loss: [0.12605015]
Epoch: 20 Loss: [0.12253361]
Epoch: 21 Loss: [0.11974452]
Epoch: 22 Loss: [0.11716044]
Epoch: 23 Loss: [0.1152773]
Epoch: 24 Loss: [0.11359722]
Epoch: 25 Loss: [0.11218452]
Epoch: 26 Loss: [0.11089571]
Epoch: 27 Loss: [0.10998049]
Epoch: 28 Loss: [0.10936147]
Epoch: 29 Loss: [0.1085618]
Epoch: 30 Loss: [0.10789888]
Epoch: 31 Loss: [0.10762097]
Epoch: 32 Loss: [0.10729053]
Epoch: 33 Loss: [0.10702001]
Epoch: 34 Loss: [0.10661078]
Epo

Epoch: 293 Loss: [0.10618122]
Epoch: 294 Loss: [0.10583019]
Epoch: 295 Loss: [0.10613163]
Epoch: 296 Loss: [0.10618686]
Epoch: 297 Loss: [0.10611059]
Epoch: 298 Loss: [0.10604237]
Epoch: 299 Loss: [0.10591933]
Epoch: 300 Loss: [0.10585351]
Epoch: 301 Loss: [0.10621739]
Epoch: 302 Loss: [0.10593543]
Epoch: 303 Loss: [0.10614888]
Epoch: 304 Loss: [0.10607988]
Epoch: 305 Loss: [0.10567458]
Epoch: 306 Loss: [0.10632984]
Epoch: 307 Loss: [0.10610002]
Epoch: 308 Loss: [0.10597859]
Epoch: 309 Loss: [0.10619326]
Epoch: 310 Loss: [0.10612725]
Epoch: 311 Loss: [0.10571836]
Epoch: 312 Loss: [0.10638162]
Epoch: 313 Loss: [0.10615196]
Epoch: 314 Loss: [0.10609096]
Epoch: 315 Loss: [0.10597385]
Epoch: 316 Loss: [0.10618963]
Epoch: 317 Loss: [0.10612955]
Epoch: 318 Loss: [0.10571737]
Epoch: 319 Loss: [0.10638986]
Epoch: 320 Loss: [0.10616403]
Epoch: 321 Loss: [0.10604865]
Epoch: 322 Loss: [0.10626471]
Epoch: 323 Loss: [0.10620759]
Epoch: 324 Loss: [0.10615253]
Epoch: 325 Loss: [0.10573754]
Epoch: 326

In [7]:
z1 = relu(np.matmul(W1,test_x[4]).reshape(-1,1)+b1)
z2 = relu(np.matmul(W2,z1).reshape(-1,1)+b2)
out = softmax(np.matmul(W3,z2).reshape(-1,1) + b3)

In [8]:
print(out)

[[1.18872252e-14]
 [9.99999864e-01]
 [1.35838655e-07]]


In [9]:
print(test_y[4])

[0. 1. 0.]


In [10]:
preds = []
true = []
# print(np.argmax(out))
for i in range(len(test_x)):
    z1 = relu(np.matmul(W1,test_x[i]).reshape(-1,1)+b1)
    z2 = relu(np.matmul(W2,z1).reshape(-1,1)+b2)
    out = softmax(np.matmul(W3,z2).reshape(-1,1) + b3)
    preds.append(np.argmax(out))
    true.append(np.argmax(test_y[i]))

In [13]:
print(out)

[[9.99999893e-01]
 [1.07438936e-07]
 [8.10283726e-41]]


In [12]:
print(true)

[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]
